# Лабораторная работа №3
## Выполнил студент группы БВТ2005 Никитин Степан

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [3]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [14]:
def prefix(line):
    pi = [0 for i in range(len(line))]      # Массив pi указывает, куда переместить указатель позиции при несовпадении символов
    j = 0
    i = 1
    while i < len(line):
        if line[i] != line[j]:
            if j == 0:
                pi[i] = 0
                i += 1
            else:
                j = pi[j - 1]
        else:
            pi[i] = j + 1
            j += 1
            i += 1
    return pi


def alg_kmp(line, subline, reg):
    n = len(line)           # Число символов в исходной строке
    m = len(subline)        # Число символов в образе

    pi = prefix(subline)
    j = 0
    i = 0

    if n < m:
        return ('\nОшибка: образ текста больше, чем сам исходный текст')

    if reg == 0:
        line = line.lower()
        subline = subline.lower()

    while i < n:
        if line[i] == subline[j]:
            i += 1
            j += 1
            if j == m:
                # Возвращает индекс первого совпадения подстроки subline в строке line
                return (f'\nОбраз найден по индексу: {i - m}')
        else:
            if j > 0:
                j = pi[j - 1]
            else:
                i += 1
                
    if i == n and j != m:
        return ('\nОбраз не найден')


text = str(input('Введите текст: '))
subtext = str(input('Введите образ текста: '))
sens = int(input('Чувствительность к тексту:\n Введите 1, если включить\n Введите 0, если не включать\n'))

print(alg_kmp(text, subtext, sens))

Введите текст: лилилось лилилась
Введите образ текста: лилила
Чувствительность к тексту:
 Введите 1, если включить
 Введите 0, если не включать
0

Образ найден по индексу: 9


#### Упрощенный алгоритм Бойера-Мура

In [13]:
def alg_bmh(line, subline, reg):
    d = defaultdict()       # Словарь смещений
    uni = set()             # Уникальные символы в образе
    n = len(line)           # Число символов в исходной строке
    m = len(subline)        # Число символов в образе

    if reg == 0:
        line = line.lower()
        subline = subline.lower()

    for i in range(m - 2, -1, -1):          # Итерации с предпоследнего символа до первого(нулевого)
        if subline[i] not in uni:           # Если символ еще не добавлен в словарь
            d[subline[i]] = m - i - 1       # Добавляем в словарь и прописываем текущее смещение этого символа
            uni.add(subline[i])

    if subline[m - 1] not in uni:           # Отдельно формируем последний символ
        d[subline[m - 1]] = m

    d['*'] = m              # Смещение для прочих символов

    if n >= m:
        i = m - 1           # Счетчик текущего проверяемого символа в строке

        while i < n:
            k = 0
            j = 0
            flBreak = False
            for j in range(m - 1, -1, -1):          # Итерации с последнего символа до первого(нулевого)
                if line[i - k] != subline[j]:
                    if j == m - 1:
                        # Смещение, если не равен последний символ образа
                        offset = d[line[i]] if d.get(line[i], False) else d['*']
                    else:
                        offset = d[subline[j]]      # Смещение, если не равен не последнй символ образа

                    i += offset                     # Смещение счетчика строки
                    flBreak = True                  # если несовпадение символа, то flBreak = True
                    break

                k += 1      # Смещение для сравниваемого символа в строке

            if not flBreak:      # Если дошли до начала образа, значит, все его символы совпали
                 return (f'\nОбраз найден по индексу: {i - k + 1}')
        else:
            return ('\nОбраз не найден')

    else:
        return ('\nОшибка: образ текста больше, чем сам исходный текст')

text = str(input('Введите текст: '))
subtext = str(input('Введите образ текста: '))
sens = int(input('Чувствительность к тексту:\n Введите 1, если включить\n Введите 0, если не включать\n'))

print(alg_bmh(text, subtext, sens))

Введите текст: большие метеоданные
Введите образ текста: данные
Чувствительность к тексту:
 Введите 1, если включить
 Введите 0, если не включать
0

Образ найден по индексу: 13


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [5]:
from queue import PriorityQueue

N = 4        # Размерность игры
SOLVED = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
start = [1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 10, 14, 15, 0]


def moves(position):
    """ Генератор сдвигов """
    zeroPosition = position.index(0)
    # Находим позицию zP в поле размером NxN, где  i - номер строки,а j - номер столбца
    i, j = divmod(zeroPosition, N)
    offsets = []
    # В зависимости от местоположения нуля, смотрим варианты, куда его можно сдвинуть
    if i > 0:
        offsets.append(-N)    # Вниз
    if i < N - 1:
        offsets.append(N)     # Вверх
    if j > 0:
        offsets.append(-1)    # Вправо
    if j < N - 1:
        offsets.append(1)     # Влево

    for offset in offsets:
        # Считаем индексы новых позиций нуля
        swap = zeroPosition + offset
        # Выводим новое состояние поля, где 0 сдвинут в одном из возможных направлений
        yield tuple(
            position[swap] if x == zeroPosition else position[zeroPosition] if x == swap else position[x] for x in range(N * N))


def parity(permutation):
    """ Проверяет четность неправильных пар """
    seen, cycles = set(), 0
    for i in permutation:
        if i not in seen:
            cycles += 1
            while i not in seen:
                seen.add(i)
                i = permutation[i]
    return (cycles + len(permutation)) % 2


class Position:
    def __init__(self, position, start_distance):
        self.position = position
        self.start_distance = start_distance


    def __lt__(self, other):
        """ Переопределяет метод less then для работы PriorityQueue """
        return self.start_distance < other.start_distance


    def __str__(self):
        return '\n'.join((N * '{:3}').format(*[i % (N * N) for i in self.position[i:]]) for i in range(0, N * N, N))



# Если количество неправильных пар четное, то такая позиция нерешаема
if parity(start) == 0:
    print('Нет решений задачи')
else:
    # Кортеж из начальной позиции
    start = tuple(start)
    p = Position(start, 0)
    # Конструктор для очереди с приоритетом
    candidates = PriorityQueue()
    candidates.put(p)
    # Множество посещенных вершин
    visited = set([p])

    came_from = {p.position: None}  # Словарь для хранения позиции, где для каждой будет определена ее предыдущая позиция

    while p.position != SOLVED:
        # Извлекаем из очереди позицию с наименьшим приоритетом
        p = candidates.get()
        # Кладем в очередь все соседние позиции. Повторяем, пока не вытащим конечную позицию из очереди
        for k in moves(p.position):
            if k not in visited:
                candidates.put(Position(k, p.start_distance + 1))
                came_from[k] = p
                visited.add(k)

    path = []
    x = p
    prev = p
    
    while p.position != start:
        # Запоминаем откуда ход
        p = came_from[p.position]
        # Запоминаем индекс нуля из текущего состояния и по этому индексу находим элемент, который сдвинули в предыдущем состоянии
        number = p.position[prev.position.index(0)]
        path.append(number)
        # Предыдущее состояние делаем текущим
        prev = p
    # Разворачиваем путь, чтобы получить путь от первого до последнего сдвига
    path.reverse()

    print(p)
    print('\n', path, '\n')
    print(x)

  1  2  3  4
  5  6  7  8
 13  9 11 12
 10 14 15  0

 [15, 14, 10, 13, 9, 10, 14, 15] 

  1  2  3  4
  5  6  7  8
  9 10 11 12
 13 14 15  0
